# Scores voor vakken voorspellen

We geven een lijst van studenten aan een docent en vragen hoe hij/zij deze studenten inschat.
De docent mag aan iedere student een score toekennen volgens zijn gevoel (niet volgens punten).
We verwachten dat een docent dit onbewust toch zal doen op basis van zijn eigen vakken.


1. Lees het bestand "studentenScores.csv" in.  Hierin zie je de subjectieve scores, gegeven door de docent.  De score kan zijn: "goed", "gemiddeld", "slecht".  We vragen ons nu af op basis van welke criteria de docent zijn scores heeft gegeven.

2. Stel daartoe een beslissingsboom op voor de score.

3. Welke vakken geeft de docent?

4. De boomstructuur is nogal complex.

Dat komt omdat de punten veel waarden kunnen aannemen.
We kunnen dit oplossen door de punten op te delen in categorieën:
 * niet geslaagd (0-9),
 * voldoening (10-13),
 * onderscheiding (14-15),
 * grote onderscheiding (16-20).

Probeer de scores zo in te delen.


5. Bereken nu de boomstructuur opnieuw.  Is dit duidelijker?

6. Hoe zou de docent iemand inschatten als die volgende punten heeft: 9, 14, 13, 12, 15, 13, 7, 10, 18

7. Maak eens een boomstructuur met het C4.5 algoritme en een ScitKit DecisionTree op de originele gegevens en kijk welk resultaat daar uit komt.

8. Maak eens een boomstructuur met een ScitKit DecisionTree op de onehot encoded gegevens (gebruik Pandas `get_dummies`) en kijk welk resultaat daar uit komt.